In [ ]:
!export LC_ALL=en_US.UTF-8
!CUDA_VISIBLE_DEVICES=2
import sys
import tensorflow as tf
import json
from pprint import pprint
import re
import numpy as np
from embedding import *
import nltk
import itertools
import random
np.random.seed(0)

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
get_available_gpus()

In [ ]:
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.python.layers.core import Dense

In [ ]:
data = json.load(open('../train-v1.1.json'))

In [ ]:
from nltk.corpus import stopwords
#nltk.download('stopwords')
nltkStopWords = stopwords.words('english')

In [ ]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [ ]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [ ]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
        start,end = find_sub_list(answer,passage)
        if start == -1:
            continue
        a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1
    

In [ ]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [ ]:
wordsToTake = 80000
words = findKMostFrequentWords(400000)

In [ ]:
len(words)

In [ ]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


PAD_TOKEN = _add_word_reduced(PAD_WORD)
UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
    if(len(reduced_glove) == wordsToTake):
        break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

In [ ]:
!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i], end = ' ', sep = ' ')

In [ ]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

In [ ]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

In [ ]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])


In [ ]:
examples_to_take_train = 72000

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [ ]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1

In [ ]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

In [ ]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)

In [ ]:
print(reduced_glove.shape)

In [ ]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(PAD_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
        if(j<max_len):
            X_indices[i,j] = look_up_word_reduced(END_WORD)
    return X_indices

In [ ]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [ ]:
document_tokens[0]

In [ ]:
answer_labels.shape

In [ ]:
import math

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    start = 0
    while start < len(inputs[0]):
        end = min(len(inputs[0]), start + batch_size)
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels':[],
                    'answer_mask': [],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[],
                 }
        
        for index,inp in enumerate(inputs):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            
            if index == 0:
                output['document_tokens'].append(inp[start:end,0:maxD])
            elif index==1:
                output['document_lengths'].append(inp[start:end])
            elif index==2:
                output['answer_labels'].append(inp[start:end,0:maxD])
            elif index==3:
                output['answer_mask'].append(inp[start:end,0:maxA,0:maxD])
            elif index==4:
                output['answer_lengths'].append(inp[start:end])
            elif index==5:
                output['question_input_tokens'].append(inp[start:end, 0:maxQ])
            elif index==6:
                output['question_output_tokens'].append(inp[start:end, 0:maxQ])
            elif index==7:
                output['question_lengths'].append(inp[start:end])
            
        output["document_tokens"] = np.array(output["document_tokens"])
        output["document_lengths"] = np.array(output["document_lengths"])
        output["answer_labels"] = np.array(output["answer_labels"])
        output["answer_mask"] = np.array(output["answer_mask"])
        output["answer_lengths"] = np.array(output["answer_lengths"])
        output["question_input_tokens"] = np.array(output["question_input_tokens"])
        output["question_output_tokens"] = np.array(output["question_output_tokens"])
        output["question_lengths"] = np.array(output["question_lengths"])
        outputs.append(output)
        start = start + batch_size
            
    return outputs


In [ ]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths]
                    ,batch_size)

for b in batch_input:
    for k, v in b.items():
        b[k] = v.squeeze(0)

In [ ]:
print("No of features:",len(batch_input[0]))
print("No of batches:",len(batch_input))

In [ ]:
split_ratio = 0.8
split = int(len(batch_input) * split_ratio)
batch_input_train = batch_input[0:split]
batch_input_test = batch_input[split:]
print("Number of train batches = ", len(batch_input_train))
print("Number of test batches = ", len(batch_input_test))

In [ ]:
class HelperFunctions:
    def __init__(self):
        None
    def printDoc(self, batch, batch_num, example_num):
        for i in batch[batch_num]['document_tokens'][example_num]:
            print(look_up_token_reduced(i),sep=" ", end=" ")
        print(" ")

    def printQues(self, batch, batch_num, example_num):
        for i in batch[batch_num]['question_output_tokens'][example_num]:
            print(look_up_token_reduced(i),sep=" ", end=" ")
        print(" ")
        
    def printGroundTruthFromBatch(self, batch, batchNum, exampleNum, printGroundTruthQuestion = True):
        print("Comprehension : ")
        self.printDoc(batch, batchNum ,exampleNum)
        print("*****************************************************************************************************")
        if printGroundTruthQuestion:
            print("Ground Truth Question : ")
            self.printQues(batch, batchNum, exampleNum)
            print("*****************************************************************************************************")
        print("Ground Truth Answer: ") #TODO FIX THIS !!!!
        print(X_train_ans_shuffled[batch_size*batchNum + exampleNum])
        print("*****************************************************************************************************")
    

In [ ]:
class FileUtils:
    def __init__(self, filepath):
        self.filepath = filepath
    def clearFile(self):
        open(self.filepath, 'w').close()
    def appendString(self, string):
        with open(self.filepath, "a") as myfile:
            myfile.write(string)

In [ ]:
class InferenceUtils:
    def __init__(self, batch_size, qgen):
        self.batch_size = batch_size
        self.qgen = qgen
        
        
    def createTokens(self, passage, batch_size):
        passage = word_tokenize(passage.lower())
        passage_len = len(passage)
        p = np.array(sentences_to_indices_glove([passage], passage_len))
        p_len = np.array(passage_len).reshape((1,))
        p_batch = np.repeat(p,repeats=batch_size, axis = 0)
        p_len_batch = np.repeat(p_len,repeats=batch_size, axis = 0)
        return p_batch, p_len_batch
    
    def makeInferenceOnText2(self, passage, batch_size, use_beam = False):
    
        p_batch, p_len_batch = self.createTokens(passage, batch_size)
        answers = session.run(qgen.answer_tags, {
            qgen.d_tokens: p_batch,
            qgen.d_lengths: p_len_batch,
        })
        answers = np.argmax(answers[0], 1)
        print("Answers = ", answers)
        answers = np.nonzero(answers)[0]
        
        outputs = []
        for i in np.split(answers, np.where(np.diff(answers) != 1)[0]+1):
            print(i)
            left,right = i[0],i[-1]+1
            answer = " ".join(word_tokenize(passage.lower())[j] for j in range(left,right))
            self.makeInferenceOnText(passage, answer, session, batch_size, use_beam)
            
    
    def makeInferenceOnText(self, passage, answer, batch_size, use_beam = False):
        passage = word_tokenize(passage.lower())
        passage_len = len(passage)

        answer = word_tokenize(answer.lower())
        answer_len = len(answer)    

        print("Passage Length = ", passage_len, ", Answer Length = ",answer_len)

        left,right = find_sub_list(answer,passage)
        if(left==-1):
            print("Couldn't find answer in the passage !!")
            return
        p = np.array(sentences_to_indices_glove([passage], passage_len))

        p_len = np.array(passage_len).reshape((1,))
        a_len = np.array(answer_len).reshape((1,))

        ans_labels = np.zeros((1,passage_len))
        ans_labels[0][left:right] = 1

        enc_mask = np.zeros((1,answer_len, passage_len))
        for i in range(left,right):
            enc_mask[0,i-left,i] = 1

        p_batch = np.repeat(p,repeats=batch_size, axis = 0)
        p_len_batch = np.repeat(p_len,repeats=batch_size, axis = 0)
        a_labels_batch = np.repeat(ans_labels,repeats=batch_size, axis = 0)
        enc_mask_batch = np.repeat(enc_mask,repeats=batch_size, axis = 0)
        a_len_batch = np.repeat(a_len,repeats=batch_size, axis = 0)
        
        output = {'document_tokens':p_batch,
                    'document_lengths':p_len_batch,
                    'answer_labels':a_labels_batch,
                    'answer_mask': enc_mask_batch,
                    'answer_lengths': a_len_batch
                 }
        if use_beam:
            qgen.inferFromBatchBeam([output], 0, 1, 5, shuffle = False, printGroundTruthQuestion = False)
        else: 
            qgen.inferFromBatch([output], 0, 1, shuffle = False, printGroundTruthQuestion = False)
            
    # Takes as input -> batch of labels and outputs in matrix form, gives mean bleu score for the batch
    def bleu_score_batch(labels, outputs):
        batch_bleu_score = 0
        #np.where(hypothesis =='<EOS>')[0][0]
        for i in range(labels.shape[0]):
            gt = ' '.join(labels[i]).replace('<END>','')
            out = ' '.join(outputs[i][1:]).replace('<END>','')
            batch_bleu_score += nltk.translate.bleu_score.sentence_bleu(gt, out)
        return(batch_bleu_score/labels.shape[0])
    
    # Takes as input 2 sentences and outputs the bleu score
    def blue_score(label, output):
        return(nltk.translate.bleu_score.sentence_bleu(label,output))


In [ ]:
class QGen:
    def __init__(
        self, learning_rate, cell_size , n_layers, reduced_glove, batch_size, 
            attention_type = 'Luong', cell_type = 'GRU', sess=tf.Session(), 
            helperFunctions = HelperFunctions(), fileUtils = FileUtils('/data/ra2630/tfLog64k'),
            grad_clip=1.0, beam_width=10, force_teaching_ratio=0.5):
        self.learning_rate = learning_rate
        self.cell_size = cell_size
        self.cell_type = cell_type
        self.attention_type = attention_type
        self.n_layers = n_layers
        self.grad_clip = grad_clip
        self.glove_embedding = reduced_glove
        self.batch_size = batch_size
        self.beam_width = beam_width
        self.force_teaching_ratio = force_teaching_ratio
        self.sess = sess
        self.helperFunctions = helperFunctions
        self.fileUtils = fileUtils
        self.build_graph()
        
    def saveSession(self, filepath):
        saver = tf.train.Saver()
        saver.save(self.sess, filepath)
        
    def loadFromSession(self, filepath):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        session = tf.Session(config=config)
        saver = tf.train.Saver()
        saver.restore(self.sess, filepath)


    def build_graph(self):
        self.add_embedding_layer()
        self.add_document_encoder_layer()
        self.add_answer_encoder_layer()
        with tf.variable_scope('decode'):
            self.add_decoder_for_training()
        with tf.variable_scope('decode', reuse=True):
            self.add_beam_decoder_for_inference()
        self.add_decoder_for_inference()
            
        
        self.add_backward_path()


    def add_embedding_layer(self): 
        self.embedding = tf.get_variable("embedding", initializer=self.glove_embedding)
        self. embedding = tf.cast(self.embedding, dtype=tf.float32)
        self.embedding_dimensions = reduced_glove.shape[1]
        self.vocabulary_size = reduced_glove.shape[0]
        
    def add_document_encoder_layer(self):
        self.d_tokens = tf.placeholder(tf.int32, shape=[None, None])
        self.d_lengths = tf.placeholder(tf.int32, shape=[None])
        
        document_emb = tf.nn.embedding_lookup(self.embedding, self.d_tokens)
        document_emb = tf.cast(document_emb, dtype=tf.float64)
        
        if self.cell_type == 'LSTM':
            self.document_forward_cell = tf.contrib.rnn.LSTMCell(self.cell_size)
            self.document_backward_cell = tf.contrib.rnn.LSTMCell(self.cell_size)
        elif self.cell_type == 'GRU':
            self.document_forward_cell = tf.contrib.rnn.GRUCell(self.cell_size)
            self.document_backward_cell = tf.contrib.rnn.GRUCell(self.cell_size)
        
        self.document_encoder_outputs, self.document_encoder_states = tf.nn.bidirectional_dynamic_rnn(
            self.document_forward_cell, 
            self.document_backward_cell, 
            document_emb, 
            self.d_lengths, 
            dtype=tf.float64
        )
        
        self.document_encoder_outputs = tf.concat(self.document_encoder_outputs, 2)
        self.document_encoder_outputs = tf.cast(self.document_encoder_outputs,tf.float32)
        self.answer_tags = tf.layers.dense(inputs = self.document_encoder_outputs, units=2)


        self.a_labels = tf.placeholder(tf.int32, shape=[None, None])
        self.answer_mask = tf.sequence_mask(self.d_lengths, dtype=tf.float32)
        self.answer_loss = seq2seq.sequence_loss(
            logits=self.answer_tags, targets=self.a_labels, weights=self.answer_mask)
        
        self.answer_loss = tf.Print(self.answer_loss, [self.answer_loss], message="This is answer_loss: ")
        


    def add_answer_encoder_layer(self):
        self.answer_encoder_input_mask = tf.placeholder(
        tf.float32, shape=[None, None, None])
        
        self.answer_encoder_inputs = tf.matmul(self.answer_encoder_input_mask, self.document_encoder_outputs)
        self.answer_encoder_lengths = tf.placeholder(tf.int32, shape=[None])

        if self.cell_type == 'GRU':
            self.answer_encoder_cell = tf.contrib.rnn.GRUCell(self.document_forward_cell.state_size + self.document_backward_cell.state_size)
            self.question_decoder_cell = tf.contrib.rnn.GRUCell(self.answer_encoder_cell.state_size)
        
        elif self.cell_type == 'LSTM':
            self.answer_encoder_cell = tf.contrib.rnn.LSTMCell(self.document_forward_cell.state_size[0] + self.document_backward_cell.state_size[0])
            self.question_decoder_cell = tf.contrib.rnn.LSTMCell(self.answer_encoder_cell.state_size[0])
            
        _, self.answer_encoder_state = tf.nn.dynamic_rnn(
            self.answer_encoder_cell, self.answer_encoder_inputs, self.answer_encoder_lengths, dtype=tf.float32)



    def add_attention_for_training(self):
        if self.attention_type == 'Luong':
            self.attention_function = tf.contrib.seq2seq.LuongAttention
        else:
            self.attention_function = tf.contrib.seq2seq.BahdanauAttention
            
        if self.cell_type == 'GRU':
            attention_mechanism = self.attention_function(
                num_units = self.answer_encoder_cell.state_size, 
                memory = self.document_encoder_outputs,
                memory_sequence_length = self.d_lengths)

            self.question_decoder_cell_attention = tf.contrib.seq2seq.AttentionWrapper(
                cell = self.question_decoder_cell, 
                attention_mechanism = attention_mechanism,
                attention_layer_size = self.question_decoder_cell.state_size)

        elif self.cell_type == 'LSTM':
            attention_mechanism = self.attention_function(
                num_units = self.answer_encoder_cell.state_size[0], 
                memory = self.document_encoder_outputs,
                memory_sequence_length = self.d_lengths)

            self.question_decoder_cell_attention = tf.contrib.seq2seq.AttentionWrapper(
                cell = self.question_decoder_cell, 
                attention_mechanism = attention_mechanism,
                attention_layer_size = self.question_decoder_cell.state_size[0])
        
        self.question_decoder_cell_attention_without_beam = self.question_decoder_cell_attention


    def add_decoder_for_training(self):
        self.add_attention_for_training()
        self.question_decoder_inputs = tf.placeholder(tf.int32, shape=[None, None])
        self.question_decoder_lengths = tf.placeholder(tf.int32, shape=[None])

        question_decoder_embedding = tf.nn.embedding_lookup(self.embedding, self.question_decoder_inputs)
        question_decoder_embedding = tf.cast(question_decoder_embedding,tf.float32)

        helper = seq2seq.TrainingHelper(question_decoder_embedding , self.question_decoder_lengths)

        self.projection = Dense(self.vocabulary_size, use_bias=False)
        
        decoder = seq2seq.BasicDecoder(
            self.question_decoder_cell_attention, 
            helper, 
            self.question_decoder_cell_attention.zero_state(self.batch_size, dtype=tf.float32).clone(
              cell_state=self.answer_encoder_state),
            output_layer = self.projection
        )
        
        decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder)
        self.training_question_decoder_outputs = decoder_outputs.rnn_output
        
    def add_decoder_for_inference(self):
        helper = seq2seq.GreedyEmbeddingHelper(
            self.embedding, 
            tf.fill([self.batch_size], START_TOKEN), 
            END_TOKEN
        )
        decoder = seq2seq.BasicDecoder(
            self.question_decoder_cell_attention_without_beam, 
            helper, 
            self.question_decoder_cell_attention_without_beam.zero_state(self.batch_size, dtype=tf.float32).clone(
                  cell_state=self.answer_encoder_state), output_layer=self.projection)
        
        decoder_outputs, _, _ = seq2seq.dynamic_decode(decoder, maximum_iterations=max_question_len)
        self.decoder_outputs = decoder_outputs.rnn_output
        
        


    def add_beam_attention_for_inference(self):
        self.document_encoder_output_tiled = tf.contrib.seq2seq.tile_batch(self.document_encoder_outputs, self.beam_width)
        self.answer_encoder_state_tiled = tf.contrib.seq2seq.tile_batch(self.answer_encoder_state, self.beam_width)
        self.document_lengths_tiled = tf.contrib.seq2seq.tile_batch(self.d_lengths, self.beam_width)

        if self.cell_type == 'GRU':
            attention_mechanism = self.attention_function(
                num_units = self.answer_encoder_cell.state_size, 
                memory = self.document_encoder_output_tiled,
                memory_sequence_length = self.document_lengths_tiled)

            self.question_decoder_cell_attention = tf.contrib.seq2seq.AttentionWrapper(
                cell = self.question_decoder_cell, 
                attention_mechanism = attention_mechanism,
                attention_layer_size = self.question_decoder_cell.state_size)
            

        elif self.cell_type == 'LSTM':
            attention_mechanism = self.attention_function(
                num_units = self.answer_encoder_cell.state_size[0], 
                memory = self.document_encoder_output_tiled,
                memory_sequence_length = self.document_lengths_tiled)

            self.question_decoder_cell_attention = tf.contrib.seq2seq.AttentionWrapper(
                cell = self.question_decoder_cell, 
                attention_mechanism = attention_mechanism,
                attention_layer_size = self.question_decoder_cell.state_size[0])



    def add_beam_decoder_for_inference(self):
        self.add_beam_attention_for_inference()
        
        decoder = tf.contrib.seq2seq.BeamSearchDecoder(
            cell = self.question_decoder_cell_attention,
            embedding = self.embedding,
            start_tokens = tf.fill([self.batch_size], START_TOKEN),
            end_token = END_TOKEN,
            initial_state = self.question_decoder_cell_attention.zero_state(self.batch_size * self.beam_width,tf.float32).clone(
                cell_state = self.answer_encoder_state_tiled),
            beam_width = self.beam_width,
            output_layer = self.projection,
            length_penalty_weight = 0.0)
        
        
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
            decoder = decoder,
            maximum_iterations = max_question_len
        )
        self.predicting_question_ids = predicting_decoder_output.predicted_ids[:, :, :]


    def add_backward_path(self):
        self.decoder_labels = tf.placeholder(tf.int64, shape=[None, None])
        question_mask = tf.sequence_mask(self.question_decoder_lengths ,dtype=tf.float32)
        self.question_loss = seq2seq.sequence_loss(
            logits = self.training_question_decoder_outputs, 
            targets = self.decoder_labels,
            weights = question_mask)
        self.question_loss = tf.Print(self.question_loss, [self.question_loss], message="This is question_loss: ")
        
        self.net_loss = self.question_loss + self.answer_loss
        
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        gvs = optimizer.compute_gradients(self.net_loss)
        capped_gvs = [(tf.clip_by_value(grad, -self.grad_clip, self.grad_clip), var) for grad, var in gvs]
        self.train_op = optimizer.apply_gradients(capped_gvs)


    def fit(self, batch_input_train, batch_input_test,save_after_every_epoch = 5, n_epoch=60, generateTrain = False, generateTest = False, clearFile = False):

        self.sess.run(tf.global_variables_initializer())
        if clearFile:
            self.fileUtils.clearFile()
        
        for epoch in range(1, n_epoch + 1):
            batch_loss = 0
            for batchNum in range(len(batch_input_train)):
                t = session.run([self.train_op, self.net_loss, self.question_loss, self.answer_loss], {
                    self.d_tokens: batch_input_train[batchNum]['document_tokens'],
                    self.d_lengths: batch_input_train[batchNum]['document_lengths'],
                    self.a_labels: batch_input_train[batchNum]['answer_labels'],
                    self.answer_encoder_input_mask: batch_input_train[batchNum]['answer_mask'],
                    self.answer_encoder_lengths: batch_input_train[batchNum]['answer_lengths'],
                    self.question_decoder_inputs: batch_input_train[batchNum]['question_input_tokens'],
                    self.decoder_labels: batch_input_train[batchNum]['question_output_tokens'],
                    self.question_decoder_lengths: batch_input_train[batchNum]['question_lengths']
                })

                print("Batch: {0}/{1}, Loss: {2}".format(batchNum, len(batch_input_train), t[1]), end='\r')
                #self.fileUtils.appendString("Batch: {0}/{1}, Loss: {2}".format(batchNum, len(batch_input_train), t[1]))
                batch_loss += t[1]
                sys.stdout.flush()
            batch_loss /= len(batch_input_train)
            print("Batch - Loss (Train): {0}".format(batch_loss))
            #self.fileUtils.appendString("Batch - Loss (Train): {0} Epoch - {1}\n".format(batch_loss, epoch))("Batch - Loss: {0} Epoch - {1}\n".format(batch_loss, epoch))
            sys.stdout.flush()
            
            batch_loss = 0
            for batchNum in range(len(batch_input_test)):
                t = session.run([self.net_loss, self.question_loss, self.answer_loss], {
                    self.d_tokens: batch_input_test[batchNum]['document_tokens'],
                    self.d_lengths: batch_input_test[batchNum]['document_lengths'],
                    self.a_labels: batch_input_test[batchNum]['answer_labels'],
                    self.answer_encoder_input_mask: batch_input_test[batchNum]['answer_mask'],
                    self.answer_encoder_lengths: batch_input_test[batchNum]['answer_lengths'],
                    self.question_decoder_inputs: batch_input_test[batchNum]['question_input_tokens'],
                    self.decoder_labels: batch_input_test[batchNum]['question_output_tokens'],
                    self.question_decoder_lengths: batch_input_test[batchNum]['question_lengths']
                })
                batch_loss += t[1]
            batch_loss /= len(batch_input_test)
            print("Batch - Loss (Test): {0}".format(batch_loss))
            #self.fileUtils.appendString("Batch - Loss (Test): {0} Epoch - {1}\n".format(batch_loss, epoch))
            sys.stdout.flush()
            if generateTrain:
                print("Train Samples Generated : ")
                self.inferFromBatch(batch_input_train, random.randint(0,len(batch_input_train)-1), 5)
                self.inferFromBatchBeam(batch_input_train, random.randint(0,len(batch_input_train)-1), 5, 5)
            if generateTest:
                self.inferFromBatch(batch_input_test, random.randint(0,len(batch_input_test)-1), 5)
                self.inferFromBatchBeam(batch_input_test, random.randint(0,len(batch_input_test)-1), 5, 5)
                
            if save_after_every_epoch % epoch == 0:
                self.saveSession("/data/ra2630/tf64k")
        
    

    def inferFromBatch(self, batch, batchNum, numExamples, shuffle = True, printGroundTruthQuestion = True):
        if shuffle:
            p = random.sample(range(0, self.batch_size), self.batch_size)
        else:
            p = range(0,self.batch_size)
        j = 0
        questions = session.run(self.decoder_outputs, {
            self.d_tokens: batch[batchNum]['document_tokens'][p],
            self.d_lengths: batch[batchNum]['document_lengths'][p],
            self.a_labels: batch[batchNum]['answer_labels'][p],
            self.answer_encoder_input_mask: batch[batchNum]['answer_mask'][p],
            self.answer_encoder_lengths: batch[batchNum]['answer_lengths'][p],
        })
        p = p[:numExamples]

        qs = np.argmax(questions, 2)
        for i in p:
            question = itertools.takewhile(lambda t: t != END_TOKEN, qs[j])
            print("----------------------------------------------------------------------------------------------------")
            self.helperFunctions.printGroundTruthFromBatch(batch, batchNum, i, printGroundTruthQuestion)
            print("Generated Question :")
            print(" ".join(look_up_token_reduced(token) for token in question))
            print("*****************************************************************************************************")    
            print("----------------------------------------------------------------------------------------------------")
            j=j+1
    
    def inferFromBatchBeam(self, batch, batchNum, numExamples, numQuestions, shuffle = True, printGroundTruthQuestion = True):
        
        if shuffle:
            p = random.sample(range(0, self.batch_size), self.batch_size)
        else:
            p = range(0,self.batch_size)
        j = 0
        questions = session.run(self.predicting_question_ids, {
            self.d_tokens: batch[batchNum]['document_tokens'][p],
            self.d_lengths: batch[batchNum]['document_lengths'][p],
            self.a_labels: batch[batchNum]['answer_labels'][p],
            self.answer_encoder_input_mask: batch[batchNum]['answer_mask'][p],
            self.answer_encoder_lengths: batch[batchNum]['answer_lengths'][p],
        })
        p = p[:numExamples]

        for i in p:
            question = itertools.takewhile(lambda t: t != END_TOKEN, questions[j,:,0])
            print("----------------------------------------------------------------------------------------------------")
            self.helperFunctions.printGroundTruthFromBatch(batch, batchNum, i, printGroundTruthQuestion)
            for k in range(numQuestions):
                print("Generated Question Number : ", (k+1))
                print(" ".join(look_up_token_reduced(token) for token in question))
            print("*****************************************************************************************************")    
            print("----------------------------------------------------------------------------------------------------")
            j=j+1
        
        
        

In [ ]:
tf.reset_default_graph()
config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
session = tf.Session(config=config)

qgen = QGen(learning_rate = 3e-3,
            cell_size = 500, 
            n_layers = 1, 
            reduced_glove = reduced_glove,
            batch_size = batch_size,
            attention_type = 'Bahadanau', 
            cell_type = 'GRU', 
            sess=session, 
            grad_clip=1.0, 
            beam_width=10, 
            force_teaching_ratio=0.5)

In [ ]:
sys.stdout = open('/data/ra2630/tfLog64k', 'w', 1)
qgen.fit(
    batch_input_train, 
    batch_input_test, 
    n_epoch=30, 
    save_after_every_epoch = 5, 
    generateTrain = True, 
    generateTest = True,
    clearFile=True)

In [ ]:
qgen.loadFromSession("/data/ra2630/qgenTemp")

In [ ]:
qgen.inferFromBatch(batch_input_train, 2, 1, shuffle = False)

In [ ]:
qgen.inferFromBatchBeam(batch_input_train, 2, 1, 5, shuffle = False)

In [ ]:
inferenceUtils = InferenceUtils(batch_size=batch_size, qgen=qgen)

In [ ]:
inferenceUtils.makeInferenceOnText(
    passage = "LG Electronics has four business units: Home Entertainment, Mobile Communications, Home Appliances & Air Solutions, and Vehicle Components.", 
    answer = "Mobile Communications",
    batch_size = batch_size, 
    use_beam = True)